<a href="https://colab.research.google.com/github/abubakarkhanlakhwera/Lex-Ai/blob/main/updated_law_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit openai-whisper fpdf groq python-docx PyPDF2 streamlit-webrtc soundfile av streamlit-audiorec easyocr


In [ ]:
!pip install streamlit-audiorec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.8/872.8 kB 9.1 MB/s eta 0:00:00


In [ ]:
!pip install easyocr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 20.1 MB/s eta 0:00:00


In [ ]:
import os
import io
import tempfile
import streamlit as st
import whisper
import numpy as np
import PyPDF2
import easyocr
from PIL import Image

import soundfile as sf
import av
from fpdf import FPDF
from docx import Document
from st_audiorec import st_audiorec
from streamlit_webrtc import webrtc_streamer, AudioProcessorBase

from groq import Groq

# ---------------------------
# Application Templates Dictionary
# ---------------------------
application_options = {
    "Bail Application (Court & Litigation)": "Bail Application – Request for the release of an accused person on bail. Provide details of the case, grounds for bail, and supporting evidence.",
    "Stay Order Application (Court & Litigation)": "Stay Order Application – Request to temporarily halt legal proceedings or actions. Include reasons and any urgent circumstances.",
    "Writ Petition (Court & Litigation)": "Writ Petition – Filed in higher courts to challenge government actions. Include facts, legal basis, and desired relief.",
    "Appeal Application (Court & Litigation)": "Appeal Application – Request a higher court to review and overturn a lower court's decision. State the grounds for appeal clearly.",
    "Revision Petition (Court & Litigation)": "Revision Petition – Filed to correct a legal or factual error in a lower court’s order. Specify the error and the needed correction.",
    "FIR Application (Criminal Law)": "FIR Application – Filed with police to report a crime. Detail the incident, evidence, and identify the accused.",
    "Anticipatory Bail Application (Criminal Law)": "Anticipatory Bail Application – Request bail before an arrest occurs. Provide justification and supporting facts.",
    "Quashing of FIR Application (Criminal Law)": "Quashing of FIR Application – Filed in higher courts to dismiss false or frivolous FIRs. Explain the basis for quashing.",
    "Divorce Application (Family Law)": "Divorce Application – Filed to initiate divorce proceedings. Include grounds for divorce and details regarding custody, maintenance, etc.",
    "Child Custody Application (Family Law)": "Child Custody Application – Request legal custody of a child. Provide details on the child's welfare and parental capabilities.",
    "Maintenance Application (Family Law)": "Maintenance Application – Request for financial support from a spouse. State the amount requested and justification.",
    "Property Dispute Application (Civil Law)": "Property Dispute Application – Filed for issues regarding ownership, possession, or inheritance. Outline the dispute and evidence.",
    "Injunction Application (Civil Law)": "Injunction Application – Request to prevent a specific action (e.g., stopping construction). Explain the urgency and harm.",
    "Declaration Suit Application (Civil Law)": "Declaration Suit Application – Filed to declare a legal right or status. Specify the legal right and any disputes.",
    "Trademark Registration Application (Business & Corporate)": "Trademark Registration Application – Filed to register a trademark. Include details of the mark and its use.",
    "Company Registration Application (Business & Corporate)": "Company Registration Application – Required to legally form a business entity. Provide company details and structure.",
    "Bankruptcy Application (Business & Corporate)": "Bankruptcy Application – Filed by individuals or businesses facing insolvency. Detail the financial situation and liabilities.",
    "Lease Agreement Application (Real Estate & Property)": "Lease Agreement Application – Formalizes rental agreements. Specify terms, duration, and responsibilities.",
    "Property Transfer Application (Real Estate & Property)": "Property Transfer Application – Filed to transfer property ownership. Include property details and transfer conditions.",
    "Eviction Application (Real Estate & Property)": "Eviction Application – Request to remove a tenant from a property. Provide reasons for eviction and supporting evidence."
}

# ---------------------------
# Groq Client & Legal Prompts
# ---------------------------
def get_groq_client():
    os.environ['GROQ_API_KEY'] = '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'
    return Groq(api_key=os.getenv("GROQ_API_KEY"))

SYSTEM_PROMPT = """
You are LEX Ai, a highly specialized AI legal assistant trained in Pakistani law.
Your role is to analyze the legal problem described by the user, understand the matter they are going through, and provide a comprehensive legal solution.
Provide detailed legal reasoning, relevant citations, and practical steps tailored to the Pakistani legal context.
"""

CHAT_SYSTEM_PROMPT = """
You are LEX Ai, a highly specialized legal assistant trained in Pakistani law.
Answer legal queries clearly and concisely with accurate references.
"""

def generate_legal_draft(user_query):
    client = get_groq_client()
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query}
    ]
    response = client.chat.completions.create(
        messages=messages,
        model="llama-3.3-70b-versatile"
    )
    return response.choices[0].message.content

def generate_chat_response(user_query):
    client = get_groq_client()
    messages = [
        {"role": "system", "content": CHAT_SYSTEM_PROMPT},
        {"role": "user", "content": user_query}
    ]
    response = client.chat.completions.create(
        messages=messages,
        model="llama-3.3-70b-versatile"
    )
    return response.choices[0].message.content

# ---------------------------
# PDF & Word Generation Functions
# ---------------------------
def generate_pdf(text):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    lines = []
    for paragraph in text.split("\n"):
        while len(paragraph) > 0:
            if len(paragraph) <= 100:
                lines.append(paragraph)
                break
            else:
                split_at = paragraph.rfind(" ", 0, 100)
                if split_at == -1:
                    split_at = 100
                lines.append(paragraph[:split_at])
                paragraph = paragraph[split_at:].strip()
    for line in lines:
        pdf.cell(0, 10, txt=line, ln=1)
    pdf_str = pdf.output(dest="S")
    return pdf_str.encode("latin1")

def generate_word(text):
    document = Document()
    for paragraph in text.split("\n"):
        document.add_paragraph(paragraph)
    word_buffer = io.BytesIO()
    document.save(word_buffer)
    word_data = word_buffer.getvalue()
    word_buffer.close()
    return word_data

# ---------------------------
# Whisper Transcription Function
# ---------------------------
def transcribe_audio(audio_bytes):
    model = whisper.load_model("base")
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
        tmp.write(audio_bytes)
        tmp_filename = tmp.name
    result = model.transcribe(tmp_filename, fp16=False)
    os.remove(tmp_filename)
    return result["text"]

# ---------------------------
# PDF Extraction Function
# ---------------------------
def extract_text_from_pdf(file_obj):
    try:
        reader = PyPDF2.PdfReader(file_obj)
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
        return text
    except Exception as e:
        return f"Error extracting text: {e}"

# ---------------------------
# Image Extraction Function using EasyOCR
# ---------------------------
def extract_text_from_image(image_file):
    reader = easyocr.Reader(['en'])
    image = Image.open(image_file)
    image_np = np.array(image)
    result = reader.readtext(image_np, detail=0)
    extracted_text = "\n".join(result)
    return extracted_text

# ---------------------------
# Sidebar: Simplified Conversation Type Selection
# ---------------------------
st.sidebar.markdown("**Select Type of Conversation:**")
action = st.sidebar.radio("", ["Chat with LEX Ai", "Start New Case", "Generate Custom Application"])

# ---------------------------
# Main Interface Title & Description
# ---------------------------
st.title("LEX Ai")
st.markdown("""
**LEX Ai** is your intelligent legal assistant for Pakistani law.
Harnessing the power of AI, LEX Ai understands your legal challenges and provides tailored solutions and comprehensive legal drafts.

**Select an Action** from the sidebar:
- **Chat with LEX Ai:** Ask legal questions using text, file, or voice input.
- **Start New Case:** Describe your legal problem step-by-step and receive a comprehensive legal solution.
- **Generate Custom Application:** Edit and generate a legal application using a pre-defined template.
""")

# ---------------------------
# Mode: Chat with LEX Ai
# ---------------------------
if action == "Chat with LEX Ai":
    st.header("Chat with LEX Ai")
    st.markdown("**Input Options:**")
    chat_input_mode = st.radio("Select Input Method:",
                               ["Text Input", "Upload Audio File", "Record Voice (Real-time)", "Upload Image", "Upload PDF"],
                               key="chat_input_mode")

    query_text = ""
    if chat_input_mode == "Text Input":
        query_text = st.text_area("Enter your legal query:")

    elif chat_input_mode == "Upload Audio File":
        audio_file = st.file_uploader("Upload an audio file (wav, mp3, ogg)",
                                      type=["wav", "mp3", "ogg"], key="chat_audio")
        if audio_file:
            query_text = transcribe_audio(audio_file.read())
            st.text_area("Transcribed Query", query_text, height=100)

    elif chat_input_mode == "Record Voice (Real-time)":
        st.markdown("Record your query using the button below:")
        audio_bytes = st_audiorec()
        if audio_bytes:
            st.audio(audio_bytes, format="audio/wav")
            with st.spinner("Transcribing voice input..."):
                query_text = transcribe_audio(audio_bytes)
            st.text_area("Transcribed Query", query_text, height=100)

    elif chat_input_mode == "Upload Image":
        image_file = st.file_uploader("Upload an image (png, jpg, jpeg)",
                                      type=["png", "jpg", "jpeg"], key="chat_image")
        if image_file:
            query_text = extract_text_from_image(image_file)
            st.text_area("Extracted Text from Image", query_text, height=100)

    elif chat_input_mode == "Upload PDF":
        pdf_file = st.file_uploader("Upload a PDF file",
                                    type=["pdf"], key="chat_pdf")
        if pdf_file:
            query_text = extract_text_from_pdf(pdf_file)
            st.text_area("Extracted Text from PDF", query_text, height=100)

    if st.button("Send Query"):
        if query_text.strip() != "":
            with st.spinner("Getting response from LEX Ai..."):
                chat_response = generate_chat_response(query_text)
            st.subheader("LEX Ai Response")
            st.text_area("Response", chat_response, height=300)
            # Download buttons for chat transcript
            pdf_data = generate_pdf(chat_response)
            word_data = generate_word(chat_response)
            st.download_button("Download as PDF", data=pdf_data, file_name="LEXAi_chat_response.pdf", mime="application/pdf")
            st.download_button("Download as Word", data=word_data, file_name="LEXAi_chat_response.docx",
                               mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document")
        else:
            st.warning("Please enter or provide a query.")

# ---------------------------
# Mode: Generate Custom Application
# ---------------------------
elif action == "Generate Custom Application":
    st.header("Generate Custom Application")
    with st.expander("Application Template", expanded=True):
        selected_app = st.selectbox("Select Application Type", list(application_options.keys()))
        custom_template = st.text_area("Edit Application Template", value=application_options[selected_app], height=150)
    if st.button("Generate Legal Draft from Custom Application"):
        with st.spinner("Generating legal draft..."):
            legal_draft = generate_legal_draft(custom_template)
            st.session_state.legal_draft = legal_draft
        st.success("Draft generated successfully!")
    if "legal_draft" in st.session_state and st.session_state.legal_draft:
        st.subheader("Generated Legal Draft (2000 words with citations)")
        st.text_area("Legal Draft", st.session_state.legal_draft, height=600)
        pdf_data = generate_pdf(st.session_state.legal_draft)
        word_data = generate_word(st.session_state.legal_draft)
        st.download_button("Download as PDF", data=pdf_data, file_name="LEXAi_legal_draft.pdf", mime="application/pdf")
        st.download_button("Download as Word", data=word_data, file_name="LEXAi_legal_draft.docx",
                           mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document")

# ---------------------------
# Mode: Start New Case (Legal Solution Mode)
# ---------------------------
elif action == "Start New Case":
    st.header("Start New Case - Get Your Legal Solution with LEX Ai")

    # Define wizard steps focused on problem understanding and legal solution
    wizard_steps = ["Problem Overview", "Case Facts", "Legal Analysis", "Proposed Legal Solution", "Review and Generate"]

    # Initialize wizard index in session state if not already set
    if 'wizard_index' not in st.session_state:
        st.session_state.wizard_index = 0

    # Get current step and corresponding key
    current_index = st.session_state.wizard_index
    current_step = wizard_steps[current_index]
    step_key = current_step.replace(" ", "_").lower()

    st.subheader(f"Step {current_index + 1} of {len(wizard_steps)}: {current_step}")

    # Helper function to handle input based on selected method
    def handle_input(step_key):
        input_method = st.radio(
            "Select Input Method:",
            ["Text Input", "Upload PDF", "Upload Image", "Record/Upload Audio"],
            key=f"input_method_{step_key}"
        )

        # Text input
        if input_method == "Text Input":
            user_input = st.text_area("Enter your text below:",
                                      value=st.session_state.get(step_key, ""),
                                      height=150)
            st.session_state[step_key] = user_input

        # Upload PDF file
        elif input_method == "Upload PDF":
            pdf_file = st.file_uploader("Upload a PDF file", type=["pdf"], key=f"pdf_{step_key}")
            if pdf_file is not None:
                extracted_text = extract_text_from_pdf(pdf_file)
                st.session_state[step_key] = extracted_text
            user_input = st.text_area("Extracted Text (edit as needed):",
                                      value=st.session_state.get(step_key, ""),
                                      height=150)
            st.session_state[step_key] = user_input

        # Upload Image file
        elif input_method == "Upload Image":
            image_file = st.file_uploader("Upload an image (png, jpg, jpeg)", type=["png", "jpg", "jpeg"], key=f"image_{step_key}")
            if image_file is not None:
                extracted_text = extract_text_from_image(image_file)
                st.session_state[step_key] = extracted_text
            user_input = st.text_area("Extracted Text (edit as needed):",
                                      value=st.session_state.get(step_key, ""),
                                      height=150)
            st.session_state[step_key] = user_input

        # Record or Upload Audio
        elif input_method == "Record/Upload Audio":
            audio_option = st.radio("Audio Input Method:", ["Upload Audio File", "Record Voice (Real-time)"], key=f"audio_option_{step_key}")
            if audio_option == "Upload Audio File":
                audio_file = st.file_uploader("Upload an audio file (wav, mp3, ogg)", type=["wav", "mp3", "ogg"], key=f"audio_file_{step_key}")
                if audio_file:
                    with st.spinner("Transcribing audio..."):
                        transcription = transcribe_audio(audio_file.read())
                    st.session_state[step_key] = transcription
            else:
                st.markdown("Record your audio using the button below:")
                audio_bytes = st_audiorec(key=f"audio_record_{step_key}")
                if audio_bytes:
                    st.audio(audio_bytes, format="audio/wav")
                    with st.spinner("Transcribing voice input..."):
                        transcription = transcribe_audio(audio_bytes)
                    st.session_state[step_key] = transcription

            user_input = st.text_area("Transcribed Text (edit as needed):",
                                      value=st.session_state.get(step_key, ""),
                                      height=150)
            st.session_state[step_key] = user_input

    # For all steps except the final "Review and Generate" step, display input handler
    if current_step != "Review and Generate":
        handle_input(step_key)

        # Navigation buttons in two columns
        col1, col2 = st.columns(2)
        with col1:
            if current_index > 0 and st.button("Previous Step"):
                st.session_state.wizard_index -= 1
                st.rerun()
        with col2:
            if st.button("Next Step"):
                st.session_state.wizard_index += 1
                st.rerun()

    # Final step: Review combined input and generate legal solution
    else:
        full_input = (
            f"Problem Overview:\n{st.session_state.get('problem_overview', '')}\n\n"
            f"Case Facts:\n{st.session_state.get('case_facts', '')}\n\n"
            f"Legal Analysis:\n{st.session_state.get('legal_analysis', '')}\n\n"
            f"Proposed Legal Solution:\n{st.session_state.get('proposed_legal_solution', '')}"
        )
        st.text_area("Final Combined Input", full_input, height=300)

        if st.button("Generate Legal Solution"):
            with st.spinner("Generating legal solution..."):
                legal_draft = generate_legal_draft(full_input)
                st.session_state.legal_draft = legal_draft
            st.success("Legal solution generated successfully!")

        if "legal_draft" in st.session_state and st.session_state.legal_draft:
            st.subheader("Generated Legal Solution")
            st.text_area("Legal Solution", st.session_state.legal_draft, height=600)
            pdf_data = generate_pdf(st.session_state.legal_draft)
            word_data = generate_word(st.session_state.legal_draft)
            st.download_button("Download as PDF", data=pdf_data, file_name="LEXAi_legal_solution.pdf", mime="application/pdf")
            st.download_button("Download as Word", data=word_data, file_name="LEXAi_legal_solution.docx",
                               mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document")


2025-02-22 13:48:54.192 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 13:48:54.196 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 13:48:54.199 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 13:48:54.202 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 13:48:54.205 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 13:48:54.209 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 13:48:54.212 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 13:48:54.215 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
!wget -qO- ipv4.icanhazip.com

34.80.133.64


In [ ]:
!streamlit run app.py & npx  localtunnel --port 8501

y


⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.133.64:8501

⠼⠴your url is: https://big-symbols-care.loca.lt
2025-02-22 18:58:35.047 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2025-02-22 18:58:35.293 Examining the path of torch.classes raised:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 345, in run
    if asyncio.get_running_loop().is_running():
       ^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: no running event loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/watcher/local_sources_watcher.py", line 217, in get_modul